In [1]:
# Modified from https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

In [2]:
lr = 0.1

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [4]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


In [6]:
net = None
net = VGG('VGG19')
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
print(net)

DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

In [8]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
            
        if not batch_idx % 50:
                print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
                
        #progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            if not batch_idx % 50:
                print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

            #progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                #% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving at {}%'.format(acc))
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [9]:
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    test(epoch)


Epoch: 0
0 391 Loss: 2.604 | Acc: 10.156% (13/128)
50 391 Loss: 3.701 | Acc: 10.202% (666/6528)
100 391 Loss: 3.126 | Acc: 10.620% (1373/12928)
150 391 Loss: 2.875 | Acc: 10.782% (2084/19328)
200 391 Loss: 2.739 | Acc: 10.735% (2762/25728)
250 391 Loss: 2.653 | Acc: 10.657% (3424/32128)
300 391 Loss: 2.593 | Acc: 10.631% (4096/38528)
350 391 Loss: 2.550 | Acc: 10.635% (4778/44928)
0 100 Loss: 2.235 | Acc: 13.000% (13/100)
50 100 Loss: 2.281 | Acc: 11.765% (600/5100)
Saving at 12.11%

Epoch: 1
0 391 Loss: 2.300 | Acc: 7.031% (9/128)
50 391 Loss: 2.281 | Acc: 11.504% (751/6528)
100 391 Loss: 2.272 | Acc: 12.260% (1585/12928)
150 391 Loss: 2.260 | Acc: 13.266% (2564/19328)
200 391 Loss: 2.227 | Acc: 14.513% (3734/25728)
250 391 Loss: 2.198 | Acc: 15.466% (4969/32128)
300 391 Loss: 2.165 | Acc: 16.302% (6281/38528)
350 391 Loss: 2.137 | Acc: 16.920% (7602/44928)
0 100 Loss: 1.888 | Acc: 21.000% (21/100)
50 100 Loss: 1.939 | Acc: 23.078% (1177/5100)
Saving at 22.62%

Epoch: 2
0 391 Loss: 1

In [9]:
net.load_state_dict(torch.load("checkpoint/ckpt.pth")['net'])

# Send model to GPU
if device:
    net.cuda()
    
net.eval()
net

DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=0

In [10]:
for batch_idx, (features, targets) in enumerate(testloader):

    features = features
    targets = targets
    break

In [11]:
features.shape, targets.shape

(torch.Size([100, 3, 32, 32]), torch.Size([100]))

In [12]:
targets

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7])

In [13]:
logits = net(features.to(device))
print(logits.shape)
_, predicted_labels = torch.max(logits, 1)
predicted_labels.shape

torch.Size([100, 10])


torch.Size([100])

In [14]:
predicted_labels

tensor([3, 8, 8, 8, 6, 6, 1, 2, 3, 9, 3, 9, 4, 5, 8, 8, 5, 3, 8, 6, 9, 0, 0, 9,
        4, 4, 4, 2, 9, 6, 6, 5, 6, 2, 9, 3, 4, 9, 9, 5, 0, 6, 3, 6, 0, 9, 3, 9,
        4, 2, 9, 8, 3, 3, 8, 8, 4, 3, 3, 3, 7, 3, 6, 3, 6, 6, 8, 0, 3, 7, 2, 6,
        8, 8, 8, 2, 9, 3, 5, 8, 8, 8, 1, 5, 5, 9, 3, 3, 8, 9, 0, 6, 8, 6, 4, 6,
        6, 0, 2, 3], device='cuda:0')

In [15]:
preds = predicted_labels.cpu().data.numpy()
preds.shape

(100,)

In [16]:
labels = targets.cpu().data.numpy()
labels.shape

(100,)

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

accuracy_score(preds, labels)

0.6

In [25]:
from sklearn.metrics import classification_report


target_names = list(classes)
print(classification_report(labels, preds, target_names=target_names))

              precision    recall  f1-score   support

       plane       0.57      0.40      0.47        10
         car       1.00      0.33      0.50         6
        bird       0.29      0.25      0.27         8
         cat       0.42      0.80      0.55        10
        deer       0.38      0.43      0.40         7
         dog       0.43      0.38      0.40         8
        frog       0.81      0.81      0.81        16
       horse       1.00      0.18      0.31        11
        ship       0.72      1.00      0.84        13
       truck       0.71      0.91      0.80        11

    accuracy                           0.60       100
   macro avg       0.63      0.55      0.53       100
weighted avg       0.66      0.60      0.57       100



In [20]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits = model(features)
        _, predicted_labels = torch.max(logits, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [21]:
# Compute test accuracy
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(net, testloader)))

Test accuracy: 61.39%
